In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sys, os
scr_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(scr_dir)


from load import TimeSeriesLoader
import warnings
warnings.filterwarnings('ignore')
# import preprocessing
from preprocess import (
    create_rolling_ts, 
    split_data, 
    flatten
)

from error_metrics import regression_metrics


## Import data from archive

In [2]:
ROOT_PATH = 'C://Users/gilbe/Documents/aifi-bootcamp'
df = pd.read_csv(f'{ROOT_PATH}/data/aapl.csv')
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.set_index('Unnamed: 0', inplace=True)
df.index.rename('Date', inplace=True)
# df.rename(columns={'Unnamed: 0', 'Date'}, inplace=True)
df.head()

ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.

In [ ]:
df.hist(figsize=(10, 4));
df.plot(figsize=(10, 4), subplots=True);

# Create train-val-test

In [ ]:
data_train, data_val, data_test = split_data(df[['4. close']], train_size=.75, valid_size=.125)

xtrain, ytrain = create_rolling_ts(input_data=data_train, lookback=30, return_np_array=True)
xval, yval = create_rolling_ts(input_data=data_val, lookback=30, return_np_array=True)
xtest, ytest = create_rolling_ts(input_data=data_test, lookback=30, return_np_array=True)


# Flatten data to N X 1 dimensions
xtrain, ytrain = flatten(xtrain), flatten(ytrain)
xval, yval = flatten(xval), flatten(yval)
xtest, ytest = flatten(xtest), flatten(ytest)

In [ ]:
xtrain.shape, ytrain.shape

In [ ]:
xtrain.reshape(xtrain.shape[0], -1).shape, ytrain.reshape(ytrain.shape[0], -1).shape

In [ ]:
# xt, yt = train_test_split(data_train, 30)

In [ ]:
regressor = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=800, 
    max_depth=5
)


regressor.fit(xtrain.reshape(xtrain.shape[0], -1), ytrain.reshape(ytrain.shape[0], -1))



In [ ]:
pred_train = regressor.predict(xtrain)
pred_val = regressor.predict(xval)
mse_train, r2_train, mae_train, smape_train = regression_metrics(ytrain, pred_train)
mse_val, r2_val, mae_val, smape_val = regression_metrics(yval, pred_val)
print('mse-train:', mse_train, 'mse-val:', mse_val)
print('r2-train:', r2_train, 'r2-val:', r2_val)
print('smape-train:', smape_train, 'smape-val:', smape_val)


df_train = pd.DataFrame(
    {
        'ytrain': ytrain.reshape(-1), 
        'pred_train': pred_train.reshape(-1), 
    }
).plot(figsize=(10, 5), grid=True);

df_train = pd.DataFrame(
    {
         'yval': yval.reshape(-1), 
         'pred_val': pred_val.reshape(-1)
        }
).plot(figsize=(10, 5), grid=True);